<a href="https://colab.research.google.com/github/Shreyankthehacker/Learning-awee/blob/main/Learning%20-%20Awee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install uv
!uv pip install -r requirements.txt

In [ ]:
from pydantic import BaseModel,Field
from typing import List,Annotated,operator

class State(BaseModel):
    query : str = Field(description='The query given by the user')
    answer : Annotated[List[str],operator.add] = Field(description="Answer given by the application for better understanding of the user")


from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()


llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash')


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

another_example_prompt = '''
You are given an example of a structured document or response.

Your task is to generate another example that follows the same structure, tone, and formatting — but with different content, keeping it logically consistent and realistic.

Instructions:
1. Carefully analyze the style, format, and data points in the provided example.
2. Generate a new example that could logically follow or be used as a sibling example.
3. Maintain the same level of detail and type of content.
4. Do not copy or paraphrase — create original data with the same schema.

Example:
{query}

Now, generate one more example in the exact same structure.
'''


template = ChatPromptTemplate.from_template(another_example_prompt)

example_chain = template | llm | StrOutputParser()

def get_example(state:State):
    result = example_chain.invoke({'query':state.query})
    return {'answer':state.answer+[result]}

In [ ]:
similar_question = '''
You are given a question. Your task is to generate 3 similar questions that ask about the same topic or intent but are phrased differently.

Guidelines:
1. Preserve the core meaning of the question.
2. Use different wording, sentence structure, or question style (e.g., descriptive, interrogative, comparative).
3. Keep the tone and difficulty level consistent.
4. Avoid exact synonyms in all versions — be creative in rephrasing.

Example:
Original Question: What are the applications of soft matter physics?

Similar Questions:
1. How is soft matter physics applied in real-world scenarios?
2. What are some practical uses of soft matter in industry and research?
3. In which fields is soft matter physics commonly utilized?

Now, generate 3 similar questions for the following:

Original Question:{query}
'''

template = ChatPromptTemplate.from_template(similar_question)

simq_chain = template | llm | StrOutputParser()

def get_similar_questions(state:State):
    result = simq_chain.invoke({'query':state.query})
    return {'answer':state.answer+[result]}

In [ ]:
from serpapi import GoogleSearch # type: ignore
import os
from dotenv import load_dotenv
load_dotenv()

SERP = os.getenv('SERPAPI_API_KEY')

def youtube_search_serpapi(state:State):
    print("yt")
    params = {
        "engine": "youtube",
        "search_query": state.query,
        "api_key":SERP
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    videos = results.get("video_results", [])[:5]
    print(videos)
    formatted_results = [
        f"{i + 1}. {v.get('title')}\n   {v.get('link')}"
        for i, v in enumerate(videos)
    ]

    return {'answer':state.answer+["Youtube videos that you can go through are"]+[formatted_results]}




In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults()

def bookrec(state: State):
    '''Tool to perform web search for getting relevant books'''
    print('Running web search...')

    # Ask the LLM to create a search query


from serpapi import GoogleSearch

def bookrec(state: State):
    params = {
        "engine": "google",
        "q":f"{state.query} related boks",
        "api_key": SERP,    # your SerpAPI key as a string or env var
        "num": 5           # get top 5 results
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    organic_results = results.get("organic_results", [])[:5]
    print(organic_results)

    formatted_results = [
        f"{i + 1}. {res.get('title')}\n   {res.get('link')}"
        for i, res in enumerate(organic_results)
    ]

    return {'answer': state.answer + ["Google search results:"] + formatted_results}


In [ ]:
import wikipedia

def wiki_explainer_tool(state):
    """Explains a topic using Wikipedia summary."""
    print("wiki")
    #result = llm.invoke({f" i want to get blogs realted to {state.query} from wen search for that get me a suitable searching query so that i can execute and get the blogs related to this query"})
    try:
        out = wikipedia.summary(state.query, sentences=5)
    except Exception as e:
       return {'answer':state.answer}
    return {'answer':state.answer+["Wikipidea information"]+[out]}


In [ ]:
from IPython.display import Video
import subprocess
import warnings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re


template_for_manim = '''
You are an expert in using Manim (Mathematical Animation Engine) to create beautiful and precise mathematical animations using Python code.

Your task is to generate Manim Python code based solely on:

1. A **math-related query** (e.g., explain Pythagoras' theorem visually, show vector addition, illustrate derivative on a graph).
2. A **simple visual description** or example for how it could be animated — only using **text** or **emojis**, no images.

⚠️ Guidelines:
- Only output valid Python code compatible with Manim Community Edition.
- Do not include any text explanation or description — return **code only**.
- Do not use images or external assets — use only text (like Tex, MathTex, MarkupText) or emojis when needed.
- Try your best to match the query and visualization using Manim's animation capabilities.

IMPORTANT: You must output **only the raw executable Python code**, with no additional text, no comments, no markdown fences (no triple backticks or 'python' tags), and no other formatting or explanation before or after the code.

At the very end of your code, add the line:
print("enjoy animation")

Write the code in such a way that the animation should be pleasing to watch with colour and make sure words don't overlap on one another.

INPUT FORMAT:
Query: {query}
Example: {example}

OUTPUT FORMAT:
Only the Python code (for Manim), exactly as it should be saved in a `.py` file, with nothing else.

Now generate the code.
'''

eg_template = prompt = (
    "You are a helpful and knowledgeable math tutor.\n"
    "I will give you either a math problem or just the name of a topic.\n"
    "If I give you a math problem, solve it step by step with clear and concise explanations.\n"
    "If I give you only a topic name, return a simple but effective example problem along with a full solution that helps explain the topic.\n"
    "Keep your responses easy to understand and focused on building intuition."

    "Input format :"
    "Query:{query}"
)

topic_chain = ChatPromptTemplate.from_template(eg_template) | llm | StrOutputParser()



prompt_manim = ChatPromptTemplate.from_template(template_for_manim)

chain = prompt_manim | llm | StrOutputParser()


import subprocess

def video_animation(state: State):
    query = state.query

    example = topic_chain.invoke({'query': query})
    result = chain.invoke({'query': query, 'example': example})

    print("Initial generated code:")
    print(result)

    marker = "from manim import *"
    sindex = result.find(marker)
    if sindex == -1:
        return {"error": "Generated code not found in output."}


    lindex = result.rfind("```")

    code = result[sindex:lindex]

    max_attempts = 3
    attempt = 0

    while attempt < max_attempts:
        with open("hello.py", "w") as f:
            f.write(code)


        process = subprocess.run(
            ['manim', '-pql', 'hello.py'],
            capture_output=True,
            text=True
        )

        if process.returncode == 0:
            print("Manim ran successfully.")
            break


        error_message = process.stderr
        print(f"Manim error on attempt {attempt+1}:\n{error_message}")


        fix_prompt = (
            f"The following Manim code failed to run with error:\n{error_message}\n\n"
            f"Please fix the code so it runs successfully: annd just return the python code that is executable and directly able to write in .py file i dont want anything additional\n\n{code}"
        )

        fixed_result = llm.invoke(fix_prompt)
        fixed_code_start = fixed_result.content.find(marker)
        if fixed_code_start == -1:

            return {"error": "LLM did not return valid fixed code.", "llm_response": fixed_result.content}
        index = fixed_result.rfind("```")

        code = fixed_result.content[fixed_code_start:index].strip()
        attempt += 1

    if attempt == max_attempts and process.returncode != 0:

        return {"error": "Failed to generate runnable Manim code after multiple attempts.", "last_error": error_message}

    return {"State": state, "message": "Manim animation generated and ran successfully."}






In [ ]:
import json
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = Chroma(persist_directory="/home/shreyank/Gen-ai/Task-3/education/chroma_course_db", embedding_function=embedding_model)

def course_recommendation(state: State):
    query = state.query
    print("searching")

    results = vectorstore.similarity_search(query, k=3)

    # Load all courses from JSON file (assuming it's a list of dicts)
    with open('course.json', 'r') as f:
        courses = json.load(f)

    # Extract course names from the top-k documents
    matched_courses = []
    for doc in results:
        course_name = doc.metadata.get("course_name")
        if course_name:
            for c in courses:
                if c.get("course_name") == course_name:
                    matched_courses.append({
                        "course_name": course_name,
                        "course_url": c.get("course_url"),
                        "abstract": c.get("abstract", ""),
                        "instructor": c.get("instructor", "")
                    })
                    break  # Break after first match to avoid duplicates

    # Build the final answer
    recommendations = ["Recommended NPTEL course(s):"]
    for course in matched_courses:
        recommendations.append(f"{course['course_name']} - {course['course_url']}")

    return {"answer": state.answer + ["Recommended NPTEL courses are"]+recommendations}


In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
from pydub import AudioSegment
import os

def record_audio_to_mp3(filename="output.mp3", duration=5, sample_rate=44100, channels=2):
    """
    Records audio from the microphone and saves it as an MP3 file.

    Args:
        filename (str): Output MP3 file name.
        duration (int): Duration of the recording in seconds.
        sample_rate (int): Sampling rate in Hz.
        channels (int): Number of audio channels (1=Mono, 2=Stereo).
    """
    wav_filename = filename.replace(".mp3", ".wav")

    print(f"🎤 Recording for {duration} seconds...")
    audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=channels)
    sd.wait()
    print("✅ Recording complete.")

    # Save temporary WAV file
    write(wav_filename, sample_rate, audio)

    # Convert to MP3
    sound = AudioSegment.from_wav(wav_filename)
    sound.export(filename, format="mp3")
    print(f"💾 Saved MP3 as: {filename}")

    # Cleanup
    os.remove(wav_filename)

# Example usage
if __name__ == "__main__":
    record_audio_to_mp3("my_recording.mp3", duration=5)


🎤 Recording for 5 seconds...
✅ Recording complete.
💾 Saved MP3 as: my_recording.mp3


In [ ]:
import whisper
import warnings
warnings.filterwarnings("ignore")

model = whisper.load_model("tiny")

def transcribe(audio):
    text = model.transcribe(audio)['text']
    print(text)
    return text


def get_query(state:State):
    choice = input("Enter the choice 1 for audio any button for text")
    if choice=='1':
        record_audio_to_mp3(duration = 50)
        question = transcribe("my_recording.mp3")
    question = input("Enter your query")
    return {'query':question}


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()


llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash')


In [ ]:
from langgraph.graph import StateGraph,START,END

builder = StateGraph(State)

builder.add_node(get_query)
builder.add_node("video_animation",video_animation)
builder.add_node("Example Generator",get_example)
builder.add_node(course_recommendation)
builder.add_node("Similar questions",get_similar_questions)
builder.add_node("book Recommendations",bookrec)
builder.add_node("Blogs",wiki_explainer_tool)
builder.add_node("Youtube recommendations",youtube_search_serpapi)

In [ ]:
builder.add_edge(START,"get_query")
builder.add_edge("get_query","video_animation")
builder.add_edge("get_query","Example Generator")
builder.add_edge("get_query","course_recommendation")
builder.add_edge("get_query","Similar questions")
builder.add_edge("get_query",'book Recommendations')
builder.add_edge("get_query",'Blogs')
builder.add_edge("get_query",'Youtube recommendations')


In [ ]:
graph = builder.compile()

In [ ]:
state = State(query = '',answer= [])
state = graph.invoke(state)

wiki
yt
searching
[{'position_on_page': 2, 'title': 'Beggar Method | Permutation Combination | Mathematics | ALLEN Digital', 'link': 'https://www.youtube.com/watch?v=W8WIgd5OjpQ', 'serpapi_link': 'https://serpapi.com/search.json?engine=youtube_video&v=W8WIgd5OjpQ', 'channel': {'name': 'ALLEN Career Institute', 'link': 'https://www.youtube.com/@ALLENCareerInstituteofficial', 'verified': True, 'thumbnail': 'https://yt3.ggpht.com/eYL1nv41lFYd7d49CW9VwIutXmWTJueXlf7d8-qUgwsNQVRwNK_nsq-UUuUkOvLxnJOEird9AWk=s68-c-k-c0x00ffffff-no-rj'}, 'published_date': '3 years ago', 'views': 18539, 'length': '4:06', 'description': '#ALLENDigital #KotaCoaching #ALLENKota #PermutationCombination #Mathematics #Learning #Education\xa0...', 'thumbnail': {'static': 'https://i.ytimg.com/vi/W8WIgd5OjpQ/hq720.jpg?sqp=-oaymwEjCOgCEMoBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCiPcjw8Ygh2kEjLmS-IHtZa---Rw'}}, {'position_on_page': 3, 'title': 'Permutation & Combination | One Shot | #BounceBack Series | JEE Math

In [ ]:
state['query']

'Beggars problem permutation and combination'

In [ ]:
for i in state['answer']:
    print(i)
    print('\n')

Okay, I understand. I need the "Beggars problem permutation and combination" example to be provided first. Once I have that, I will generate a new example that mirrors its structure, tone, and formatting, but with entirely different content, while remaining logically consistent and realistic within the context of permutation and combination problems. I will ensure it doesn't copy or paraphrase the original example.

**Please provide the "Beggars problem permutation and combination" example.**


Here are 3 similar questions for the original question "Beggars problem permutation and combination":

1. How can permutation and combination techniques be used to solve problems involving distributing items to beggars?
2. What combinatorial approaches are effective for determining the number of ways to allocate resources among a group of recipients, similar to the classic "beggars" problem?
3. Suppose you have a certain number of identical items to distribute among a group of people. Explain ho